# **Предобработка данных для прогнозирования цен на недвижимость**

### Скачивание данных и импортирование библиотек

In [1]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

# sklearn models
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, KFold

In [2]:
# Скачиваем "сырой" датасет
!wget -O data_wo_targets.csv "https://drive.google.com/uc?export=download&id=1Fp6szjUwhLgGQktcnDAYV71Fwi-2ywOX"
clear_output()

In [104]:
# Загрузим данные в переменную X
X = pd.read_csv("data_wo_targets.csv")

### Заполнение пропусков - метод


In [105]:
# Заполнение пропусков
def fill_empty_gaps(data: pd.DataFrame, param: float) -> pd.DataFrame:
  incomplete_columns = data.columns[data.isna().any()].tolist()

  for i in incomplete_columns:
    data[i] = data[i].fillna(param)
  return data

Количество пропусков и их процентное соотношение

In [ ]:
insufficient_columns = X.columns[X.isna().any()].tolist()
for i in insufficient_columns:
  print(f'Столбец: {i}\nПустых строк: {X[i].isna().sum()}\nВсего строк: {len(X[i])}\nПроцентное соотношение: {(X[i].isna().sum()/len(X[i]))*100:.2f}%\n')

### Подготовка параметров

Фиксация seed для повторяемости

In [108]:
random_state = 42
np.random.seed(random_state)

Набор данных `y` не доступен, поэтому для проверки кода создаем его искусственно.

In [ ]:
prices = np.random.randint(low=8e5, high=1e7, size=len(X), dtype=int)
y = pd.DataFrame(
    {"id": X["id"], # Без данного стоблца не получится объединить X и y
     "price_doc": prices})

y.head(5)

Функция для предобработки данных

In [110]:
# Список столбцов с типом "object"
object_columns = [i for i, k in X.dtypes.items() if k == 'object']


def preparation(data: pd.DataFrame) -> pd.DataFrame:
    """
    Выходные параметры:
      data: pd.DataFrame
        Исходные табличные данные формата pandas.

    Возвращаемые элементы:
        Предобработанные данные формата pandas.
    """

    id = data["id"]
    data = fill_empty_gaps(data, 0) # заполнение пропусков
    data = data.drop(["id", *object_columns], axis=1) # удаление лишних столбцов

    data["id"] = id
    return data

### Сохранение и демонстрация датафрейма

Сохранение данных в файл "data.csv"

In [ ]:
X = preparation(X)
X.to_csv("data.csv", index=False)
X_y = pd.merge(X, y, on='id')

assert len(X_y) == len(y), \
    "Что-то не так с `id`!"

X = X_y.drop(["id", "price_doc"], axis=1)
X

Убедиться, что блокнот запускается без ошибок и код тренировки выполняется правильно.

In [ ]:
# Примерное время выполнения ячейки - 2-3 мин. На Яндекс.Контесте ~2-4 мин.

scores = cross_validate(RandomForestRegressor(n_estimators=5, random_state=random_state), X, y,
                        cv=KFold(n_splits=5, shuffle=True, random_state=random_state),
                        scoring=("r2"),
                        return_train_score=True)

scores

### Дополнительно

Всем объектам поменять строки на числа, так как почти у всех всего два признака

In [ ]:
object_columns = [i for i, k in X.dtypes.items() if k == 'object']
for i in object_columns:
  X[i] = X[i].astype('category')
  X[i] = X[i].cat.codes

Так как в `ecology` больше двух признаков, можно разделить на столбцы

In [ ]:
X = pd.get_dummies(X, columns=['ecology'])